In [ ]:
!pip uninstall -y triton
!pip install triton

Found existing installation: triton 3.1.0
Uninstalling triton-3.1.0:
  Successfully uninstalled triton-3.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers accelerate datasets bertviz umap-learn seaborn

# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from datasets import Dataset, DatasetDict
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
! pip install datasets
from datasets import load_dataset
from transformers import AutoTokenizer

imdb_data = load_dataset("imdb")

# Select only 500 samples for training and 500 for testing
train_subset = imdb_data["train"].shuffle(seed=42).select(range(5000))
test_subset = imdb_data["test"].shuffle(seed=42).select(range(5000))

# Initialize the tokenizer
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=256)

# Apply the tokenization to the selected subsets
train_subset = train_subset.map(tokenize, batched=True)
test_subset = test_subset.map(tokenize, batched=True)

# Prepare the model
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=2)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}


In [ ]:
training_args = TrainingArguments(
    output_dir="bert_imdb_sentiment",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=True
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=test_subset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-04407cfdf172>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.162200,0.280831,0.899600,0.899467


TrainOutput(global_step=624, training_loss=0.28800471127033234, metrics={'train_runtime': 279.485, 'train_samples_per_second': 35.78, 'train_steps_per_second': 2.233, 'total_flos': 1312397944135680.0, 'train_loss': 0.28800471127033234, 'epoch': 1.9952})

In [ ]:
!zip -r bert_imdb_sentiment.zip bert_imdb_sentiment
from google.colab import files
files.download("bert_imdb_sentiment.zip")

  adding: bert_imdb_sentiment/ (stored 0%)
  adding: bert_imdb_sentiment/training_args.bin (deflated 51%)
  adding: bert_imdb_sentiment/config.json (deflated 49%)
  adding: bert_imdb_sentiment/tokenizer_config.json (deflated 75%)
  adding: bert_imdb_sentiment/vocab.txt (deflated 53%)
  adding: bert_imdb_sentiment/model.safetensors (deflated 7%)
  adding: bert_imdb_sentiment/special_tokens_map.json (deflated 42%)
  adding: bert_imdb_sentiment/checkpoint-624/ (stored 0%)
  adding: bert_imdb_sentiment/checkpoint-624/training_args.bin (deflated 51%)
  adding: bert_imdb_sentiment/checkpoint-624/config.json (deflated 49%)
  adding: bert_imdb_sentiment/checkpoint-624/tokenizer_config.json (deflated 75%)
  adding: bert_imdb_sentiment/checkpoint-624/optimizer.pt (deflated 15%)
  adding: bert_imdb_sentiment/checkpoint-624/vocab.txt (deflated 53%)
  adding: bert_imdb_sentiment/checkpoint-624/model.safetensors (deflated 7%)
  adding: bert_imdb_sentiment/checkpoint-624/trainer_state.json (deflated 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save the trained model and tokenizer
trainer.save_model("bert_imdb_sentiment")
tokenizer.save_pretrained("bert_imdb_sentiment")

('bert_imdb_sentiment/tokenizer_config.json',
 'bert_imdb_sentiment/special_tokens_map.json',
 'bert_imdb_sentiment/vocab.txt',
 'bert_imdb_sentiment/added_tokens.json',
 'bert_imdb_sentiment/tokenizer.json')

In [15]:
from google.colab import files

# Download the zip file
files.download('/content/bert_imdb_sentiment.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>